In [12]:
import pandas as pd
import numpy as np
import h5py
import cmapPy
import cmapPy.pandasGEXpress.write_gctx as write_gctx
from cmapPy.pandasGEXpress.parse_gctx import parse
from importlib import reload
import cmapPy.pandasGEXpress.GCToo as GCToo


In [4]:
test1 = cmapPy.pandasGEXpress.parse_gctx.parse('/home/bsiranos/projects/cmapR/tests/testthat/test_n5x10.gctx')
print(test1)

cmapPy.pandasGEXpress.write_gctx.write(test1, '/home/bsiranos/test1.gctx')
test2 = cmapPy.pandasGEXpress.parse_gctx.parse('/home/bsiranos/test1.gctx')

print(test1.meth_df)
print(test2.meth_df)

b'mGCTX1.0'
src: /home/bsiranos/projects/cmapR/tests/testthat/test_n5x10.gctx
meth_df: [5 rows x 10 columns]
cov_df: [5 rows x 10 columns]
row_metadata_df: [5 rows x 0 columns]
col_metadata_df: [10 rows x 0 columns]
cid         1         2         3         4         5         6         7  \
rid                                                                         
1    0.413253 -0.848010 -1.192238  0.201875  0.085363  0.418502  0.306778   
2   -0.304028  0.717581  1.496278 -0.633134  0.762708  0.949611  1.423488   
3   -0.353891  0.357976  0.016063 -0.966516 -1.085235  0.212001 -0.928770   
4    1.688939  1.369050 -0.029643 -0.433559  0.032954  1.429242 -1.066902   
5    0.038399  0.578928  1.024624  1.822458  0.951131  0.906120 -1.899356   

cid         8         9        10  
rid                                
1    0.441256 -0.348707 -1.523495  
2   -0.797290  1.220535 -1.553791  
3   -0.082590 -0.855964 -1.508178  
4    1.451474 -0.021609  0.603870  
5    1.272806  0.208444  0.4

In [33]:
df1 = pd.DataFrame([[1, 2], [3, 4]], columns=['A', 'B'], index=['a', 'b'])
df2 = pd.DataFrame([[5, 6], [7, 8]], columns=['A', 'B'], index=['a', 'b'])
df3 = pd.concat([df1, df2], keys=('df1','df2'))
print(df1.shape)
print(df2.shape)
print(df3.shape)

(2, 2)
(2, 2)
(4, 2)


In [25]:
full_path = '/home/bsiranos/projects/cmapR/tests/testthat/test_n5x10.gctx'
data_node = "/0/DATA/0/matrix"

gctx_file = h5py.File(full_path, "r")
data_dset = gctx_file[data_node]
print(data_dset.shape)
total_rows = 5
total_cols = 10
ridx = list(range(5))
cidx = list(range(10))

(2, 10, 5)


In [30]:
"""
Parses in data_df from hdf5, subsetting if specified.

Input:
    -data_dset (h5py dset): HDF5 dataset from which to read data_df
    -ridx (list): list of indexes to subset from data_df
        (may be all of them if no subsetting)
    -cidx (list): list of indexes to subset from data_df
        (may be all of them if no subsetting)
    -row_meta (pandas DataFrame): the parsed in row metadata
    -col_meta (pandas DataFrame): the parsed in col metadata
"""

if len(ridx) == total_rows and len(cidx) == total_cols:  # no subset
    data_array = np.empty(data_dset.shape, dtype=np.float32)
    data_dset.read_direct(data_array)
    meth_array = data_array[0, :, :].transpose()
    cov_array = data_array[1, :, :].transpose()
else:
    # We can only subset on a single dimension at a time with h5py.
    # For the first dimension to use, pick the one that minimizes
    # the size of the intermediate array.
    row_first_count = total_cols * len(ridx)
    col_first_count = total_rows * len(cidx)

    if row_first_count < col_first_count:
        first_subset = data_dset[:, :, ridx].astype(np.float32)
        meth_array = first_subset[0, cidx, :].transpose()
        cov_array = first_subset[1, cidx, :].transpose()
    else:
        first_subset = data_dset[cidx, :].astype(np.float32)
        meth_array = first_subset[0, :, ridx].transpose()
        cov_array = first_subset[1, :, ridx].transpose()

# make DataFrame instance
meth_df = pd.DataFrame(meth_array, index=row_meta.index[ridx], columns=col_meta.index[cidx])
cov_df = pd.DataFrame(cov_array, index=row_meta.index[ridx], columns=col_meta.index[cidx])


NameError: name 'row_meta' is not defined

In [49]:
m1 = np.array([[1,2,3],[4,5,6]])
m2 = np.array([[11,12,13],[14,15,16]])
m3 = np.array([m1,m2])

print(m3.shape)

(2, 2, 3)


In [3]:
import cmapPy.pandasGEXpress.parse_gctx as parse_gctx
import cmapPy.pandasGEXpress.write_gctx as write_gctx
import cmapPy.pandasGEXpress.GCToo as GCToo
import pandas as pd


col_meta = parse_gctx.get_column_metadata("/home/bsiranos/projects/cmapPy/cmapPy/pandasGEXpress/tests/functional_tests/mini_gctx_with_metadata_n2x3.gctx")
row_meta = parse_gctx.get_row_metadata("/home/bsiranos/projects/cmapPy/cmapPy/pandasGEXpress/tests/functional_tests/mini_gctx_with_metadata_n2x3.gctx")
meth_df = pd.DataFrame([[-0.283359, 0.011270], [0.304119, 1.921061], [0.398655, -0.144652]],
                            index=["200814_at", "218597_s_at", "217140_s_at"],
                            columns=["LJP005_A375_24H:DMSO:-666", "LJP005_A375_24H:BRD-K76908866:10"])

cov_df = pd.DataFrame([[-0.283359, 0.011270], [0.304119, 1.921061], [0.398655, -0.144652]],
                            index=["200814_at", "218597_s_at", "217140_s_at"],
                            columns=["LJP005_A375_24H:DMSO:-666", "LJP005_A375_24H:BRD-K76908866:10"])

test_new = parse_gctx.parse('~/loyal_local/data/')

In [15]:
col_meta

chd,bead_batch,bead_revision
cid,,
LJP005_A375_24H:DMSO:-666,b19,r2
LJP005_A375_24H:BRD-K76908866:10,b19,r2


In [18]:
gt = GCToo.GCToo(meth_df, cov_df, row_meta, col_meta)

In [20]:
write_gctx.write(gt, '/home/bsiranos/projects/cmapPy/cmapPy/pandasGEXpress/tests/functional_tests/mini_gctx_with_metadata_n2x3.gctx')

In [2]:
import cmapPy.pandasGEXpress.mini_gctoo_for_testing as mini_gctoo_for_testing
import cmapPy.pandasGEXpress.parse_gctx as parse_gctx
import cmapPy.pandasGEXpress.write_gctx as write_gctx
import cmapPy.pandasGEXpress.GCToo as GCToo

mg1 = mini_gctoo_for_testing.make()
mg2 = parse_gctx.parse("cmapPy/pandasGEXpress/tests/functional_tests/mini_gctoo_for_testing.gctx")
write_gctx.write(mg1, "cmapPy/pandasGEXpress/tests/functional_tests/mini_gctoo_for_testing.gctx")

In [3]:
mg1.row_metadata_df

rhd,count_cv,distil_nsample,distil_ss,mfc_plate_id,zmad_ref
rid,,,,,
LJP007_MCF10A_24H:TRT_CP:BRD-K93918653:3.33,14|15|14,3,9.822065,-666,population
MISC003_A375_24H:TRT_CP:BRD-K93918653:3.33,13|14|13,3,6.891521,-666,population
LJP007_MCF7_24H:TRT_POSCON:BRD-K81418486:10,13|15|14|14|15|14|14|13|14|15|15|14|14|15|14|1...,66,1.358406,-666,population
LJP007_MCF7_24H:TRT_POSCON:BRD-A61304759:10,13,2,5.548899,-666,population
LJP007_MCF7_24H:CTL_VEHICLE:DMSO:-666,13,9,3.355232,-666,population
LJP007_MCF7_24H:TRT_CP:BRD-K64857848:10,14,111111,4.837643,-666,population


In [4]:
mg2.row_metadata_df

rhd,count_cv,distil_nsample,distil_ss,mfc_plate_id,zmad_ref
rid,,,,,
LJP007_MCF10A_24H:TRT_CP:BRD-K93918653:3.33,14|15|14,3,9.822065,-666,population
MISC003_A375_24H:TRT_CP:BRD-K93918653:3.33,13|14|13,3,6.891521,-666,population
LJP007_MCF7_24H:TRT_POSCON:BRD-K81418486:10,13|15|14|14|15|14|14|13|14|15|15|14|14|15|14|1...,66,1.358406,-666,population
LJP007_MCF7_24H:TRT_POSCON:BRD-A61304759:10,13,2,5.548899,-666,population
LJP007_MCF7_24H:CTL_VEHICLE:DMSO:-666,13,9,3.355232,-666,population
LJP007_MCF7_24H:TRT_CP:BRD-K64857848:10,14,111111,4.837643,-666,population


In [1]:
import cmapPy.pandasGEXpress.subset_gctoo as subset_gctoo
import cmapPy.pandasGEXpress.mini_gctoo_for_testing as mini_gctoo_for_testing
import cmapPy.pandasGEXpress.parse_gctx as parse_gctx

test_rids = ['LJP007_MCF10A_24H:TRT_CP:BRD-K93918653:3.33', 'LJP007_MCF7_24H:CTL_VEHICLE:DMSO:-666']
test_cids = ['LJP007_MCF7_24H:TRT_POSCON:BRD-A61304759:10']
mg1 = mini_gctoo_for_testing.make()

mg3 = subset_gctoo.subset_gctoo(mg1, rid=test_rids, cid=test_cids)
mg4 = parse_gctx.parse("cmapPy/pandasGEXpress/tests/functional_tests/mini_gctoo_for_testing.gctx",
                        rid=test_rids, cid=test_cids)